So what if we started at a HACK emulator

Ainstr
Cinstr


ImpBV - imp with only bitvectors.
bits - parts


Super optimization. I can encode instructions easily to Z3 as enums

erasing PC is a big step. partial evaluation?

rom as a z3 array vs not. I suppose it has to be a Z3 array biascally.

I baulked at the idea of using a dict for my switch case. But there is a version of the z3 interpreter that does this.
My if then esle have to become z3 ite(). brutal.

So you really want to partially evaluate that.  That's really interesting that this is the relationship of the PC aware version and the non PC aware version is partialk evaluation. we stage an interpeter.

Can I make this work with nmigen?

That riscv cpu https://news.ycombinator.com/item?id=27575174

Could also attempt a BF interpeter
or FORTH?
WASM?
AVR

Could I use fetch as my symbolic execution point?


I could easily translate this spec to Coq.

There is a debate if we hsould take a unop/binop/const breakdown of computations. Is this easier?

Const = 
UnOp = Id, Not, Neg, Inc, Dec
BinOp = Add, Sub Or, And,, not all combos avaianle? nope. 

Dest = One Two Three.

It's all not worth it.


We could encode evertything into Z3
We could meta program into Z3kanren... That's intriguing. but fetch...

How can I write my interpeter such that it works in all these different situations.
python control flow constructs and assignement are insufficiently generic

It is somewhat interesting that there is some relation of my difficulties here and coq functions vs relations.
If i need to run backwards or reinterpety through z3 it feels like I'm sunk.
What is the difference between a coq relation and a dict? Ok dict is functional
Or [( Jump.Op , a + b )] . This is a value levbel relation.


We can build basic blocks. Runs of Null.
And we do have direct jumps. JMP.



SMT encoding
Work at draper on veryfing binaries
Typelevel/Value vs compiletime/runtime

take what we get vs desgin from scratch

What does specification mean?

Julia? Enough macro magic to do what you need.
nmgien?
My old python catgoery blog posts - graphviz, z3, etc.


haskell to agda

pepem keynote at popl - turing 2019

systolic
skewed data
arrow catregories - comma catgeoies

deriving compostion from a functor. You have the functor but 

hmormohpism has nothing to do with laws


symbolic execution.
Seperate the 


In [26]:
from dataclasses import dataclass
from enum import Enum       
import typing
from typing import List, Union

Comp = Enum('Comp', '''ZERO ONE NEGONE D A M NOTD NOTA NOTM NEGD NEGA NEGM INCD INCA INCM DECD DECA DECM 
                         ADD_DA ADD_DM SUB_DA SUB_DM SUB_AD SUB_MD AND_DA AND_DM OR_DA OR_DM''') 
Dest = Enum('Dest', 'NULL M D MD A AM AD AMD') 
Jump = Enum('Jump', 'NULL JGT JEQ JGE JLT JNE JLE JMP') 

@dataclass
class Ainstr:
    lit: int

@dataclass
class Cinstr:
    dest: Dest
    comp: Comp
    jump: Jump

@dataclass
class HackState:
    rA: int
    rD: int
    pc: int
    mem: List[int]

def execute(instr : Cinstr, s : HackState):
    A = s.rA
    D = s.rD
    pc = s.pc
    M = s.mem[A]
    
    if instr.comp == Comp.ZERO:
        comp = 0
    elif instr.comp == Comp.ONE:
        comp = 1
    elif instr.comp == Comp.NEGONE:
        comp = -1
    elif instr.comp == Comp.D:
        comp = D
    elif instr.comp == Comp.A:
        comp = A
    elif instr.comp == Comp.M:
        comp = M
    elif instr.comp == Comp.NOTD:
        comp = ~D
    elif instr.comp == Comp.NOTA:
        comp = ~A
    elif instr.comp == Comp.NOTM:
        comp = ~M
    elif instr.comp == Comp.NEGD:
        comp = -D
    elif instr.comp == Comp.NEGA:
        comp = -A
    elif instr.comp == Comp.NEGM:
        comp = -M
    elif instr.comp == Comp.INCD:
        comp = D + 1
    elif instr.comp == Comp.INCA:
        comp = A + 1
    elif instr.comp == Comp.INCM:
        comp = M + 1
    elif instr.comp == Comp.DECD:
        comp = D - 1
    elif instr.comp == Comp.DECA:
        comp = A - 1
    elif instr.comp == Comp.DECM:
        comp = M - 1
    elif instr.comp == Comp.ADD_DA:
        comp = D + A
    elif instr.comp == Comp.ADD_DM:
        comp = D + M
    elif instr.comp == Comp.SUB_DA:
        comp = D - A
    elif instr.comp == Comp.SUB_DM:
        comp = D - M
    elif instr.comp == Comp.SUB_AD:
        comp = A - D
    elif instr.comp == Comp.SUB_MD:
        comp = M - D
    elif instr.comp == Comp.AND_DA:
        comp = D & A
    elif instr.comp == Comp.AND_DM:
        comp = D & M
    elif instr.comp == Comp.OR_DA:
        comp = D | S
    elif instr.comp == Comp.OR_DM:
        comp = D | M
    else:
        assert False
    
    if instr.dest == Dest.NULL:
        pass
    elif instr.dest == Dest.M:
        M = comp
    elif instr.dest == Dest.D:
        D = comp
    elif instr.dest == Dest.MD:
        M = comp
        D = comp
    elif instr.dest == Dest.A:
        A = comp
    elif instr.dest == Dest.AM:
        A = comp
        M = comp
    elif instr.dest == Dest.AD:
        A = comp
        D = comp
    elif instr.dest == Dest.AMD:
        A = comp
        M = comp
        D = comp
    else:
        assert False
        
    if instr.jump == Jump.NULL:
        jump = False
    elif instr.jump == Jump.JGT:
        jump = comp > 0
    elif instr.jump == Jump.JEQ:
        jump = comp == 0
    elif instr.jump == Jump.JGE:
        jump = comp >= 0
    elif instr.jump == Jump.JLT:
        jump = comp < 0
    elif instr.jump == Jump.JNE:
        jump = comp != 0
    elif instr.jump == Jump.JLE:
        jump = comp <= 0
    elif instr.jump == Jump.JMP:
        jump = True
    else:
        assert False
    
    # we may not want to mutate here.
    # return comp, jump, dest instead?
    if jump:
        s.pc = s.rA
    else:
        s.pc = pc + 1
    
    s.mem[s.rA] = M #??? What is the order here?
    s.rA = A
    s.rD = D

    # return A, D, M, jump ?
    return s

Instr = Union[Ainstr,Cinstr]

def fetch(rom : List[Instr], s : HackState) -> Instr:
    return rom[s.pc]

def run(rom):
    # hmm. We should make 16 bit vectors.
    state = HackState(0,0,0,[0]*2**13)
    while state.pc != -1:
        instr = fetch(rom, state)
        state = execute(instr, state)
    return state


In [23]:
def jump_table(comp):
    return { 
    Jump.NULL: False,
    Jump.JGT: comp > 0,
    Jump.JEQ: comp == 0,
    Jump.JGE: comp >= 0,
    Jump.JLT: comp < 0,
    Jump.JNE: comp != 0,
    Jump.JLE: comp <= 0,
    Jump.JMP: True
    }
jump_table(7)
def comp_table(s ):
    A = s.rA
    D = s.rD
    pc = s.pc
    M = s.mem[rA]
    return {
        Comp.ZERO:  0,
        Comp.ONE: 1,
        Comp.NEGONE: -1,
        Comp.D: D,
        Comp.A: A,
        Comp.M: M,
        Comp.NOTD: ~D,
        Comp.NOTA: ~A,
        Comp.NOTM: ~M,
        Comp.NEGD: -D,
        Comp.NEGA: -A,
        Comp.NEGM: -M,
        Comp.INCD: D + 1,
        Comp.INCA: A + 1,
        Comp.INCM: M + 1,
        Comp.DECD: D - 1,
        Comp.DECA: A - 1,
        Comp.DECM: M - 1,
        Comp.ADD_DA: D + A,
        Comp.ADD_DM: D + M,
        Comp.SUB_DA: D - A,
        Comp.SUB_DM: D - M,
        Comp.SUB_AD: A - D,
        Comp.SUB_MD: M - D,
        Comp.AND_DA: D & A,
        Comp.AND_DM: D & M,
        Comp.OR_DA: D | S,
        Comp.OR_DM: D | M
    }





In [ ]:
def interp_computation(instr : Cinstr, s : HackState):
    A = s.rA
    D = s.rD
    pc = s.pc
    M = s.mem[A]
    
    if instr.comp == Comp.ZERO:
        comp = 0
    elif instr.comp == Comp.ONE:
        comp = 1
    elif instr.comp == Comp.NEGONE:
        comp = -1
    elif instr.comp == Comp.D:
        comp = D
    elif instr.comp == Comp.A:
        comp = A
    elif instr.comp == Comp.M:
        comp = M
    elif instr.comp == Comp.NOTD:
        comp = ~D
    elif instr.comp == Comp.NOTA:
        comp = ~A
    elif instr.comp == Comp.NOTM:
        comp = ~M
    elif instr.comp == Comp.NEGD:
        comp = -D
    elif instr.comp == Comp.NEGA:
        comp = -A
    elif instr.comp == Comp.NEGM:
        comp = -M
    elif instr.comp == Comp.INCD:
        comp = D + 1
    elif instr.comp == Comp.INCA:
        comp = A + 1
    elif instr.comp == Comp.INCM:
        comp = M + 1
    elif instr.comp == Comp.DECD:
        comp = D - 1
    elif instr.comp == Comp.DECA:
        comp = A - 1
    elif instr.comp == Comp.DECM:
        comp = M - 1
    elif instr.comp == Comp.ADD_DA:
        comp = D + A
    elif instr.comp == Comp.ADD_DM:
        comp = D + M
    elif instr.comp == Comp.SUB_DA:
        comp = D - A
    elif instr.comp == Comp.SUB_DM:
        comp = D - M
    elif instr.comp == Comp.SUB_AD:
        comp = A - D
    elif instr.comp == Comp.SUB_MD:
        comp = M - D
    elif instr.comp == Comp.AND_DA:
        comp = D & A
    elif instr.comp == Comp.AND_DM:
        comp = D & M
    elif instr.comp == Comp.OR_DA:
        comp = D | S
    elif instr.comp == Comp.OR_DM:
        comp = D | M
    else:
        assert False
    return comp


def interp_dest(instr : Cinstr, s : HackState):
    A = s.rA
    D = s.rD
    pc = s.pc
    M = s.mem[A]

    if instr.dest == Dest.NULL:
        pass
    elif instr.dest == Dest.M:
        M = comp
    elif instr.dest == Dest.D:
        D = comp
    elif instr.dest == Dest.MD:
        M = comp
        D = comp
    elif instr.dest == Dest.A:
        A = comp
    elif instr.dest == Dest.AM:
        A = comp
        M = comp
    elif instr.dest == Dest.AD:
        A = comp
        D = comp
    elif instr.dest == Dest.AMD:
        A = comp
        M = comp
        D = comp
    else:
        assert False
        
    s = copy(s)
    s.mem[s.rA] = M #??? What is the order here?
    s.rA = A
    s.rD = D
    return s


    

In [ ]:
def z3_switch(x, d):
    acc = BoolVal(True)
    for k,v in d.items()
        acc = If(x == k, v, acc)
    return acc

# if instead of using extended if elif cases. I had used the dicctionary method
# lots of unnecessary computation. but more directly overloadable.

def switch(x,d):
    for k,v in d.items():
        if x == k:
            return v

In [ ]:
def z3_fetch(rom, state):
    while s.check():
        s.push()
        m = s.getmodel()
        mpc = m[state.pc].as_long()
        yield rom[mpc]
        s.add(state.pc != mpc)



Abstract interpetation to figure out CFG ahead of time. otherwise we have trouble.

A D PC = BitSet(2^16)
Not totally insane?
Mem is insane though.


https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8805076

It should be extremely rare there are many addresses it can jump to
And it is in fact limitted by the small program memory space.


I suspect that PC constraints can be dealth with similarly to invaraints. User can specify what possibilities and that is easy enough to verify.


In [8]:
x = set()
type(x) is set

True

In [ ]:
class SetDomain():
    def __init__(self, x):
        if type(x) is set:
            self.x = x
        elif type(x) is SetDomain:
            self.x = x.x
        else:
            self.x = set(x)
    def __add__(self,b):
        set([ x + y for x in self.x for y in b.x ])
    
    
        

In [ ]:
def alu(instr, HackState):
    If(instr == 0, )

In [3]:
class VSA():
    def __init__(self, ):
        
    def __add__():
        
        

8192

"Everything is a loop" in assembly

That's an interesting mantra. And we need invariants annotated everywhere
And straight line if the else structures are the exception rather than the rule. That allow us to propagate state summaries

An idea:
A block has multiple entries and exits. but is otherwise sitraght control flow.
Each should be labelled with a summary.


And a jump needs to verify B1_exit => B2_entry
And a block needs to verify forall n, B1_entry_n => wp(   )
where we put an assert on each exit.




In [ ]:
# or perhaps just have the last instruction be 
class Def:
    dest:
    comp:
    # jmp is Null.

# should check out angr and/or the fuzzing book.
    
class BasicBlock:
    defs: list[Def]
    jmp: Instr # no Null allowed though.

        
        

class BasicBlock():
    def __init__(self, code):
        self.body =
        self.jmp = 
def blocks(instrs):
    blocks = []
    defs = []
    for instr in instrs:
        if instr.jmp != Jump.NULL:
            blocks.append(Block(defs,instr))
            defs = []
        else:
            defs.append(instr)
    assert length(defs) == 0
    return blocks